In [148]:
class rayleigh_vapor:
    """Generates a vapor dataset from interpolated Rayleigh data"""
    def __init__(self,vdc_name=None,varnames=None,varfiles=None, rayleigh_root=None, vapor_root=None, nxyz=None, grid_file=None):
        self.numts=len(varfiles)
        self.varnames=varnames
        self.vdc_name=vdc_name
        self.nvars=len(varnames)
        self.varfiles=varfiles
        varstring=' '
        for i in range(self.nvars):
            varstring=varstring+varnames[i]
            if (i != self.nvars-1):
                varstring=varstring+':'
        self.varstring=varstring
        print(self.varstring)
        print(self.numts)
        self.vapor_root=vapor_root
        self.rayleigh_root=rayleigh_root
        self.nxyz=nxyz
        self.grid_file=grid_file
    def create_vdc(self):
        res=str(self.nxyz)
        cube_string = res+'x'+res+'x'+res
        cmd0 = 'rm '+self.vdc_name+' && '
        cmd1 = 'export PATH=$PATH:'+self.vapor_root+'/bin'
        cmd2 = ' && export LD_LIBRARY_PATH='+self.vapor_root+'/lib:$LD_LIBRARY_PATH'
        cmd3 = ' && vdccreate -dimension '+cube_string+' -numts '+str(self.numts)
        cmd3 = cmd3+' -vars3d '+self.varstring+' '+self.vdc_name
        creation_cmd=cmd1+cmd2+cmd3+' && ls'
        print(creation_cmd)
        s=os.popen(creation_cmd)
        b=s.read()
        print(b)
    def populate_vdc(self):
        for i in range(self.numts):
            for j in range(self.nvars):
                infile=self.varfiles[i][j]
                ofile=infile+'.cube'
                self.rayleigh_to_cube(infile,ofile)
                self.cube_to_vdc(ofile,i,j)
    def rayleigh_to_cube(self,infile,ofile):
        cmd1 = 'export PATH=$PATH:'+self.rayleigh_root
        cmd2 = ' && echo $PATH && interp3d -i '+infile+' -o '+ofile+' -g '+self.grid_file+' -N '+str(self.nxyz)
        cmd = cmd1+cmd2
        s=os.popen(cmd)
        b=s.read()
        print(b)
    def cube_to_vdc(self,ofile,timeind,varind):
        cmd1 = 'export PATH=$PATH:'+self.vapor_root+'/bin'
        cmd2 = ' && export LD_LIBRARY_PATH='+self.vapor_root+'/lib:$LD_LIBRARY_PATH'
        cmd3 = ' && raw2vdc -ts '+str(timeind)+' -varname '+self.varnames[varind]
        cmd3 = cmd3+' '+self.vdc_name+' '+ofile
        cmd = cmd1+cmd2+cmd3
        s=os.popen(cmd)
        b=s.read()
        print(b)
        print(cmd)
                

In [149]:
f = ['02086101_0001', '02086101_2807']
gf = '02086101_grid'
files = [f]
vdc = 'jup_test.vdc'
varnames = ['vr','vortz']
vroot='/custom/software/VAPOR3-3.2.0-Linux'
rroot='/home/feathern/devel/forks/Nick/Rayleigh/post_processing/interpolation'
cube_size=256
test = rayleigh_vapor(vdc_name=vdc,varnames=varnames,varfiles=files,vapor_root=vroot,nxyz=cube_size, grid_file=gf, rayleigh_root=rroot)
test.create_vdc()

 vr:vortz
1
export PATH=$PATH:/custom/software/VAPOR3-3.2.0-Linux/bin && export LD_LIBRARY_PATH=/custom/software/VAPOR3-3.2.0-Linux/lib:$LD_LIBRARY_PATH && vdccreate -dimension 256x256x256 -numts 1 -vars3d  vr:vortz jup_test.vdc && ls
02086101_0001
02086101_0001.cube
02086101_2807
02086101_2807.cube
02086101_grid
jup_test.vdc
test2.out
test_data
test.vdc
vapor_notebook.ipynb



In [150]:
test.populate_vdc()

/custom/software/miniconda3/envs/radev/bin:/custom/software/miniconda3/envs/radev/bin:/custom/software/miniconda3/condabin:/custom/software/mpich/3.2.1/gcc/7.3.0/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/feathern/devel/forks/Nick/Rayleigh/post_processing/interpolation


export PATH=$PATH:/custom/software/VAPOR3-3.2.0-Linux/bin && export LD_LIBRARY_PATH=/custom/software/VAPOR3-3.2.0-Linux/lib:$LD_LIBRARY_PATH && raw2vdc -ts 0 -varname vr jup_test.vdc 02086101_0001.cube
/custom/software/miniconda3/envs/radev/bin:/custom/software/miniconda3/envs/radev/bin:/custom/software/miniconda3/condabin:/custom/software/mpich/3.2.1/gcc/7.3.0/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/feathern/devel/forks/Nick/Rayleigh/post_processing/interpolation


export PATH=$PATH:/custom/software/VAPOR3-3.2.0-Linux/bin && export LD_LIBRARY_PATH=/custom/software/VAPOR3-3.2.0-Linux/li